In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from incomer.data import DataManager
import pandas as pd

filename = '../data/EURUSD60.csv'
cycles = [5, 20, 50, 100, 200, 600, 1000]
num_bars = 300
forward_bars = 48
lookbacks = 10
valid_split = 0.8

dm = DataManager(filename, cycles, num_bars=num_bars)

# x = dm.get_x(lookbacks)

xtrain, ytrain, xtest, ytest, xbase, ybase, op = dm.create_train_data(valid_split, lookbacks, forward_bars)

op = op[:-1]
xtest = xtest[:-1]

print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)

print(len(op))
print(len(xbase))
print(len(ybase))

In [ ]:
import numpy as np

def get_eq(
    self, predictions: np.array, op: np.array, verbose=1, reversed=False, rev_signal=False, min_elasped=20
):
    assert len(predictions) == len(op)
    add_balance = 50000

    trades = []
    eq = []
    store_trades = []
    entry_elasped = min_elasped

    for j, (pr, o) in enumerate(zip(predictions, op)):
        lots = round(add_balance / o)
        if reversed:
            lots = -lots
        if rev_signal:
            pr = -pr
        if pr == 1 and entry_elasped >= min_elasped and len(trades) == 0:
            if verbose:
                print(f"Entered: {o} / {lots} @ {j}")
            trades.append([o, lots])
            entry_elasped = 0
        elif pr == -1 and len(trades) > 0:
            # closed.append(pnl)
            trades = []
            if verbose:
                print(f"Closed: {o} / {lots} @ {j}")
            # entry_elasped = min_elasped
        pnl = 0
        for _, l in trades:
            pnl += (o - op[j - 1]) * l
        eq.append(pnl)
        store_trades.append(trades)
        entry_elasped += 1

    eq = np.array(eq)
    return eq


In [ ]:
from incomer.models.transformer import TradeTransformer

trader = TradeTransformer()
trader.build_model(
    input_shape=xtrain.shape[1:],
    head_size=128,
    num_heads=16,
    ff_dim=4,
    num_transformer_blocks=16,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
)
# trader.model.summary()

In [ ]:
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity

def calculate_dd(eq):
    end = np.argmax(np.maximum.accumulate(eq) - eq)
    start = np.argmax(eq[:end])
    return eq[end] - eq[start]

def get_signals(extractor, xtest, xbase, ybase):

    base = extractor.predict(xbase, verbose=0)
    predictions = extractor.predict(xtest, verbose=0)

    sim = cosine_similarity(predictions, base)

    idx = np.argmax(sim, axis=1)
    signals = ybase[idx]
    signals[signals == 2] = -1
    return signals

def get_eq(
    predictions: np.array, op: np.array, verbose=1, reversed=False, rev_signal=False, min_elasped=20
):
    assert len(predictions) == len(op)
    add_balance = 50000

    trades = []
    eq = []
    store_trades = []
    entry_elasped = min_elasped

    for j, (pr, o) in enumerate(zip(predictions, op)):
        o = o[0]
        lots = round(add_balance / o)
        if reversed:
            lots = -lots
        if rev_signal:
            pr = -pr
        if pr == 1 and entry_elasped >= min_elasped and len(trades) == 0:
            if verbose:
                print(f"Entered: {o} / {lots} @ {j}")
            trades.append([o, lots])
            entry_elasped = 0
        elif pr == -1 and len(trades) > 0:
            # closed.append(pnl)
            trades = []
            if verbose:
                print(f"Closed: {o} / {lots} @ {j}")
            # entry_elasped = min_elasped
        pnl = 0
        for _, l in trades:
            pnl += (o - op[j - 1]) * l
        eq.append(pnl)
        store_trades.append(trades)
        entry_elasped += 1

    eq = np.array(eq)
    return eq


In [ ]:
best_eq = 2
best_rev = 2

for ii in range(10):
    trader.model.fit(
        xtrain,
        ytrain,
        epochs=1,
        validation_split=0.9,
        batch_size=16,
        shuffle=True,
        verbose=0,
    )

    extractor = keras.models.Model(trader.model.input, trader.model.layers[-3].output)
    predictions = get_signals(extractor, xtest, xbase, ybase)
    eq = get_eq(predictions, op, verbose=False)
    try:
        dd = calculate_dd(eq)
    except Exception:
        continue
    if dd == 0:
        dd = 1
    score = np.sum(eq) / abs(dd)
    if score > best_eq:
        trader.model.save("data/EURUSD_best_lb30_f20.h5")
        best_eq = score
        print(f"BEST: {best_eq}")
    elif -score > best_rev:
        trader.model.save("data/EURUSD_best_lb30_f20_reverse.h5")
        best_rev = -score
        print(f"BEST REVERSE: {best_rev}")
    else:
        print(f"{ii} -> {score}")

    eq = get_eq(predictions, op, verbose=False, rev_signal=True)
    try:
        dd = calculate_dd(eq)
    except Exception:
        continue
    if dd == 0:
        dd = 1
    score = np.sum(eq) / abs(dd)
    if score > best_eq:
        trader.model.save("data/EURUSD_best_lb30_f20_revsig.h5")
        best_eq = score
        print(f"BEST: {best_eq}")
    elif -score > best_rev:
        trader.model.save("data/EURUSD_best_lb30_f20_revsig_reverse.h5")
        best_rev = -score
        print(f"BEST REVERSE: {best_rev}")
    else:
        print(f"{ii} -> {score}")


In [ ]:
score

In [ ]:
extractor = keras.models.Model(trader.model.input, trader.model.layers[-3].output)

signals = get_signals(extractor, xtest, xbase, ybase)
signals

In [ ]:
import matplotlib.pyplot as plt

eq = get_eq(signals, op)

plt.plot(np.cumsum(eq))
plt.show()